In [1]:
import os
import sys
# os.chdir('../')
sys.path.append(os.getcwd())

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.decomposition import PCA
from scipy.stats import fisher_exact, chi2_contingency
from sklearn.manifold import TSNE, MDS
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, PrecisionRecallDisplay
from multipy.fwer import sidak, hochberg
import random

from utils.viz_utils import plot_usage_matrix_pca, plot_boxplots_for_usage_matrix, _plot_feature_importance, plot_v_usage_hist, \
                                plot_cluster_map, significant_clones_distribution, plot_results_for_hla_class, plot_generated_to_real_distribution, \
                            plot_olga_cleanup_data, plot_clusters_of_clonotypes, plot_cooccurence_heatmap_with_epitopes_labeling, plot_waterfall_by_column, \
                            plot_feature_importances, SeabornFig2Grid
from utils.ml_utils import get_parameters, prepare_data, evaluate_models, split_data_by_batch, split_data, cross_validation_between_batches, make_hla_predictor
from utils.data_utils import prepare_run_column
from utils.stats_utils import evaluate_anova_testing, evaluate_mannwhitneyu_testing
from utils.clustering_utils import seqs2hamming
from utils.weblogo_utils import create_web_logo


import warnings
warnings.filterwarnings('ignore')


import importlib
imported_module = importlib.import_module("utils.viz_utils")
importlib.reload(imported_module)

<module 'utils.viz_utils' from '/home/evlasova/tcr-covid-classifier/utils/viz_utils.py'>

In [2]:
desc_for_projects_beta = pd.read_csv('data/desc_fmba_new_split_beta.csv').drop(columns=['Unnamed: 0'])[['run', 'folder']]
desc_for_projects_beta

,run,folder
0,050002290808_S117_L002,#2
1,050002630808_S113_L002,#2
2,050003120808_S112_L002,#2
3,050003130808_S114_L002,#2
4,050003450808_S118_L002,#2
...,...,...
1220,p18_72_DNA_S88_L002,#9
1221,p18_73_DNA_S89_L002,#9
1222,p18_74_DNA_S90_L002,#9
1223,p18_75_DNA_S91_L002,#9


In [4]:
desc_for_projects_alpha = pd.read_csv('data/desc_fmba_new_split_alpha.csv').drop(columns=['Unnamed: 0'])[['run', 'folder']]
desc_for_projects_alpha

,run,folder
0,050002290808_S117_L002,#2
1,050002630808_S113_L002,#2
2,050003120808_S112_L002,#2
3,050003130808_S114_L002,#2
4,050003450808_S118_L002,#2
...,...,...
1269,p18_73_DNA_S89_L002,#9
1270,p18_74_DNA_S90_L002,#9
1271,p18_75_DNA_S91_L002,#9
1272,p18_76_DNA_S92_L002,#9


In [3]:
allele_names = ['A*02', 'A*01', 'A*03', 'A*24', 'B*07', 'B*35', 'B*44', 'B*18', 'C*07', 'C*04', 'C*12', 'DQB1*05', 'DRB1*16']